In [2]:
import os
import tempfile
import unittest
import filecmp
import sbol3
import paml
import tyto
import uml
import json

from paml_convert.autoprotocol.autoprotocol_specialization import AutoprotocolSpecialization
from paml_convert.autoprotocol.strateos_api import StrateosAPI, StrateosConfig
from autoprotocol import container_type as ctype
from paml.execution_engine import ExecutionEngine
from container_api.client_api import matching_containers, strateos_id

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
## Execute a PAML protocol, emitting an Autoprotocol protocol

# Setup PAML protocol for execution
out_dir = "."
doc = sbol3.Document()
sbol3.set_namespace('https://bbn.com/scratch/')
protocol_file = os.path.join(os.getcwd(), "../test/testfiles", "igem_ludox_test.nt")
doc.read(protocol_file, 'nt')
protocol = doc.find("https://bbn.com/scratch/iGEM_LUDOX_OD_calibration_2018")
agent = sbol3.Agent("test_agent")


# Setup Output location and Credentials for Strateos
out_dir = "out"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
autoprotocol_output = os.path.join(out_dir, "test_LUDOX_autoprotocol.json")
secrets_file = os.path.join(os.getcwd(), "../secrets/strateos_secrets.json")


# Setup Autoprotocol specialization
resolutions = {
    doc.find("https://bbn.com/scratch/LUDOX") : "rs1b6z2vgatkq7",
    doc.find("https://bbn.com/scratch/ddH2O") : "rs1c7pg8qs22dt",
    "container_id" : "ct1g9qsg4wx6gcj"
}
api = StrateosAPI(cfg=StrateosConfig.from_file(secrets_file))
autoprotocol_specialization = AutoprotocolSpecialization(autoprotocol_output, api, resolutions)


# Execute the protocol
ee = ExecutionEngine(specializations=[autoprotocol_specialization])
parameter_values = [paml.ParameterValue(parameter=protocol.get_input("wavelength"), 
                                        value=uml.LiteralIdentified(value=sbol3.Measure(100, tyto.OM.nanometer)))]
execution = ee.execute(protocol, agent, id="test_execution", parameter_values=parameter_values)

# Return the instructions from the protocol
ee.specializations[0].protocol.instructions

[Instruction(provision, {'resource_id': 'rs1c7pg8qs22dt', 'measurement_mode': 'volume', 'to': [{'well': Well(Container(samples), 0, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(samples), 12, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(samples), 24, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(samples), 36, 100:microliter), 'volume': Unit(100.0, 'microliter')}]}, []),
 Instruction(provision, {'resource_id': 'rs1b6z2vgatkq7', 'measurement_mode': 'volume', 'to': [{'well': Well(Container(samples), 1, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(samples), 13, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(samples), 25, 100:microliter), 'volume': Unit(100.0, 'microliter')}, {'well': Well(Container(samples), 37, 100:microliter), 'volume': Unit(100.0, 'microliter')}]}, []),
 Instruction(spectrophotometry, {'dataref': 'measureme

In [4]:
# Display the Autoprotocol JSON
with open(autoprotocol_output, "r") as f:
    autoprotocol_json = json.loads(f.read())
autoprotocol_json

{'instructions': [{'op': 'provision',
   'resource_id': 'rs1c7pg8qs22dt',
   'measurement_mode': 'volume',
   'to': [{'well': 'samples/0', 'volume': '100:microliter'},
    {'well': 'samples/12', 'volume': '100:microliter'},
    {'well': 'samples/24', 'volume': '100:microliter'},
    {'well': 'samples/36', 'volume': '100:microliter'}]},
  {'op': 'provision',
   'resource_id': 'rs1b6z2vgatkq7',
   'measurement_mode': 'volume',
   'to': [{'well': 'samples/1', 'volume': '100:microliter'},
    {'well': 'samples/13', 'volume': '100:microliter'},
    {'well': 'samples/25', 'volume': '100:microliter'},
    {'well': 'samples/37', 'volume': '100:microliter'}]},
  {'op': 'spectrophotometry',
   'dataref': 'measurements',
   'object': 'samples',
   'groups': [{'mode': 'absorbance',
     'mode_params': {'wells': ['samples/0',
       'samples/1',
       'samples/12',
       'samples/13',
       'samples/24',
       'samples/25',
       'samples/36',
       'samples/37'],
      'wavelength': ['100:na

In [5]:
# Submit Autoprotocol object for execution at Strateos

st = api.get_strateos_connection()
response = st.analyze_run(ee.specializations[0].protocol, test_mode=True)
response

{'id': None,
 'status': 'pending',
 'title': None,
 'created_at': None,
 'updated_at': None,
 'completed_at': None,
 'conversation_id': None,
 'quote': {'items': [{'cost': '0.73',
    'quantity': 1,
    'title': 'Workcell Time',
    'run_id': None,
    'run_credit_applicable': True},
   {'cost': '3.46',
    'quantity': 1,
    'title': 'Reagents & Consumables',
    'run_id': None,
    'run_credit_applicable': False}],
  'ppp': False},
 'results': {},
 'test_mode': True,
 'accepted_by_id': 'u1b64bbpxftks',
 'accepted_at': '2021-10-19T03:45:34.329-07:00',
 'started_at': None,
 'canceled_at': None,
 'aborted_at': None,
 'predecessor_id': None,
 'draft_quote': None,
 'progress': 0,
 'protocol_id': None,
 'request_type': 'protocol',
 'launch_request_id': None,
 'flagged': False,
 'estimated_run_time_cache': None,
 'scheduled_to_start_at': None,
 'properties': {},
 'internal_run': False,
 'bsl': 1,
 'success': None,
 'success_notes': None,
 'priority': 'Medium',
 'requested_at': None,
 'total

In [ ]:
response = st.submit_run(ee.specializations[0].protocol, 
                         project_id=api.cfg.project_id,
                         title=protocol.name,
                         test_mode=True)

response